In [1]:
# define logging and working directory
from ProjectRoot import change_wd_to_project_root
change_wd_to_project_root()
from src.utils.Tensorflow_helper import choose_gpu_by_id
from pyforest import *


# define GPU id to use
# 0 = 1080 Bus ID 2
# 1 = Titan Bus ID 131
# 2 = Titan Bus ID 132
gpu_id = '0'
current_gpu = choose_gpu_by_id(gpu_id)


import logging
import json
import SimpleITK as sitk
import glob
import datetime
import random
from collections import Counter
import matplotlib.pyplot as plt
import keras
from ipywidgets import interact
%matplotlib inline
%reload_ext autoreload
%autoreload 2


from src.utils.utils_io import Console_and_file_logger, init_config
from src.visualization.visualize import show_2D_or_3D
from src.data.dataset import get_metadata_maybe, filter_4d_vol, describe_sitk, get_img_msk_files_from_split_dir
from src.data.generators import DataGenerator
from src.utils.unet_3d_metrics import weighted_dice_coefficient_loss
from src.models.ModelManager import get_model
from src.utils.KerasCallbacks import get_callbacks
from keras.utils import plot_model
import src.utils.my_metrics as metr


# define experiment name for report, model and log paths + filenames
EXPERIMENT = '3D/new_model'
now = datetime.datetime.now()

# image params
ARCHITECTURE = '3D'
DIM = [12, 224, 224]
#IMG_Z = 12
#IMG_WIDTH = 224
#IMG_HEIGHT = 224
SPACING = [1.0,1.0,8] # used by sitk, opposite order than numpy or tensorflow!
IMG_CHANNELS = 1
MASK_VALUES = [0, 1, 2, 3]  
MASK_CLASSES = len(MASK_VALUES)
# Background = 0 = Y[:,:,0]
# RV = 1 = Y[:,:,1] 
# Myo = 2 = Y[:,:,2] 
# LV = 3 = Y[:,:,3]
AUGMENT = False
SHUFFLE = True
AUGMENT_GRID = True
RESAMPLE = False
SPACING_X = 1.00
SPACING_Y = 1.00

# path params
DATASET = 'tetra'  # 'acdc' # or 'tetra'
TRAIN_PATH = 'data/raw/tetra/3D/train/'
VAL_PATH = 'data/raw/tetra/3D/val/'
TEST_PATH = 'data/raw/tetra/3D/test/'

MODEL_PATH = os.path.join(os.path.join('models', EXPERIMENT), str(now.strftime("%Y-%m-%d_%H_%M")))
TENSORBOARD_LOG_DIR = os.path.join(os.path.join('reports/tensorboard_logs', EXPERIMENT),str(now.strftime("%Y-%m-%d_%H_%M")))
CONFIG_PATH = os.path.join(os.path.join('reports/configs/',EXPERIMENT),str(now.strftime("%Y-%m-%d_%H_%M")))
HISTORY_PATH = os.path.join(os.path.join('reports/history/',EXPERIMENT),str(now.strftime("%Y-%m-%d_%H_%M")))

# training params
ARCHITECTURE = '3D'
GENERATOR_WORKER = 2 # if not set use batchsize
seed = 42
BATCHSIZE =  4 # 32, 64, 16, 1
INITIAL_EPOCH = 0
EPOCHS = 150
FOLDS = 4
EPOCHS_BETWEEN_CHECKPOINTS = 5
MONITOR_FUNCTION = 'val_dice_coef_labels'
MONITOR_MODE = 'max'

# Network params
OPTIMIZER = 'Adam'  # Adam, Adagrad, RMSprop, Adadelta,  # https://keras.io/optimizers/
ACTIVATION = 'elu'  # 'elu' --> works well with binary_crossentropy and bce_dice_loss, relu does not work, it clips negative values, bse does return negative values
LEARNING_RATE = 0.001
DECAY = 0.0
EPSILON = 1e-08
DROPOUT_L1_L2 = 0.3 # best with 0.4 and other 0.5
DROPOUT_L3_L4 = 0.4
DROPOUT_L5 = 0.5
BATCH_NORMALISATION = True

#metrics = [jaccard_coef, jaccard_coef_background, jaccard_coef_rv, jaccard_coef_lv, jaccard_coef_myo]
metrics = [
    metr.dice_coef_labels,
    metr.dice_coef_myo,
    metr.dice_coef_lv,
    metr.dice_coef_rv,
]

#LOSS_FUNCTION = keras.losses.categorical_crossentropy
#weights = np.array([1,2,2,3]) # Class one at 1, class 2, 10 times the normal weights, class 3 and 4 20x.
#LOSS_FUNCTION = weighted_categorical_crossentropy(weights)
#LOSS_FUNCTION = cce_dice_loss
LOSS_FUNCTION = metr.bce_dice_jac_loss

Console_and_file_logger(EXPERIMENT, logging.INFO)


# Define a config for param injection,
# save a serialized version, 
# make sure all paths exist
config = init_config(locals(), True)

# set warnings lvl for skimage
#warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
# define a Tensorflow config
#tf_config = tf.ConfigProto()
#tf_config.gpu_options.allow_growth = True
#tf_session = tf.Session(config=tf_config)
#tf.keras.backend.set_session(tf_session)

search for root_dir and set working directory
Working directory set to: /mnt/data/git/cardio
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1709982873418640778
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6172083567224082683
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 23137137460
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3547627308205375572
physical_device_desc: "device: 0, name: TITAN RTX, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4893802178515127108
physical_device_desc: "device: XLA_GPU device"
]


Using TensorFlow backend.
2019-09-05 12:31:08,731 INFO -------------------- Start --------------------
2019-09-05 12:31:08,732 INFO Working directory: /mnt/data/git/cardio.
2019-09-05 12:31:08,745 INFO Log file: ./logs/3D/new_model.log
2019-09-05 12:31:08,746 INFO config saved:
 {
    "ACTIVATION": "elu",
    "ARCHITECTURE": "3D",
    "AUGMENT": false,
    "AUGMENT_GRID": true,
    "BATCHSIZE": 4,
    "BATCH_NORMALISATION": true,
    "CONFIG_PATH": "reports/configs/3D/new_model/2019-09-05_12_31",
    "DATASET": "tetra",
    "DECAY": 0.0,
    "DIM": [
        12,
        224,
        224
    ],
    "DROPOUT_L1_L2": 0.3,
    "DROPOUT_L3_L4": 0.4,
    "DROPOUT_L5": 0.5,
    "EPOCHS": 150,
    "EPOCHS_BETWEEN_CHECKPOINTS": 5,
    "EPSILON": 1e-08,
    "EXPERIMENT": "3D/new_model",
    "FOLDS": 4,
    "GENERATOR_WORKER": 2,
    "HISTORY_PATH": "reports/history/3D/new_model/2019-09-05_12_31",
    "IMG_CHANNELS": 1,
    "INITIAL_EPOCH": 0,
    "LEARNING_RATE": 0.001,
    "LOSS_FUNCTION": "bce

## Get training, val and test-files

In [2]:
"""
2D special
# load only slices from the lower, middle or upper part
X_train, Y_train = get_samples(path=TRAIN_PATH, samples=0, part='all', no_patients = 0, preprocessed=False)
X_val, Y_val = get_samples(path=VAL_PATH, samples=0, part='all', no_patients = 0, preprocessed=False)
X_test, Y_test = get_samples(path=TEST_PATH, samples=0, part='all', no_patients = 0, preprocessed=False)
"""

x_train, y_train = get_img_msk_files_from_split_dir(config['TRAIN_PATH'])
x_val, y_val = get_img_msk_files_from_split_dir(config['VAL_PATH'])
x_test, y_test = get_img_msk_files_from_split_dir(config['TEST_PATH'])

logging.info('x_train files: {}, y_train files: {}'.format(len(x_train), len(y_train)))
logging.info('x_val files: {}, y_val files: {}'.format(len(x_val), len(y_val)))
logging.info('x_test files: {}, y_test files: {}'.format(len(x_test), len(y_test)))

2019-09-05 12:31:08,772 INFO x_train files: 754, y_train files: 754
2019-09-05 12:31:08,772 INFO x_val files: 130, y_val files: 130
2019-09-05 12:31:08,772 INFO x_test files: 126, y_test files: 126


## Create Datagenerator

In [3]:
# create a batch generator
batch_generator = DataGenerator(x_train, y_train, config=config)
config['AUGMENT_GRID'] = False # make sure no augmentation will be applied to the evaluation data
validation_generator = DataGenerator(x_val, y_val , config=config)
test_generator = DataGenerator(x_test, y_test, config=config)

2019-09-05 12:31:08,787 INFO Create DataGenerator
2019-09-05 12:31:08,790 INFO Datagenerator created with: 
 shape: [12, 224, 224]
 batchsize: 4
 Scaler: MinMax
 Images: 754 
 Augment_grid: True
2019-09-05 12:31:08,790 INFO No augmentation
2019-09-05 12:31:08,790 INFO Create DataGenerator
2019-09-05 12:31:08,791 INFO Datagenerator created with: 
 shape: [12, 224, 224]
 batchsize: 4
 Scaler: MinMax
 Images: 130 
 Augment_grid: False
2019-09-05 12:31:08,791 INFO No augmentation
2019-09-05 12:31:08,792 INFO Create DataGenerator
2019-09-05 12:31:08,793 INFO Datagenerator created with: 
 shape: [12, 224, 224]
 batchsize: 4
 Scaler: MinMax
 Images: 126 
 Augment_grid: False
2019-09-05 12:31:08,793 INFO No augmentation


## Visualize one batch

In [4]:
# Select batch generator output
x = ''
y = ''
@interact
def select_batch(batch = (0,len(batch_generator), 1)):
    global x, y
    x, y = batch_generator.__getitem__(batch)


interactive(children=(IntSlider(value=94, description='batch', max=188), Output()), _dom_classes=('widget-inte…

In [5]:
@interact
def select_image_in_batch(im = (0,config['BATCHSIZE']- 1, 1)):
    
    # define a different logging level to make the generator steps visible
    logging.getLogger().setLevel(logging.INFO)
    show_2D_or_3D(x[im], y[im])
    plt.show()

interactive(children=(IntSlider(value=1, description='im', max=3), Output()), _dom_classes=('widget-interact',…

In [6]:
@interact
def interact_load_pretrained_model(config_file=glob.glob('reports/configs/{}/**/**/*.json'.format(config.get('ARCHITECTURE', '2D')), recursive=False), load=False):
    """
    load past config for model training 
    """
    # load config with all params into global namespace
    if load:
        with open(config_file, encoding='utf-8') as data_file:
            config = json.loads(data_file.read())
        #globals()['MODEL_PATH'] = config['MODEL_PATH']
        logging.info('Experiment: {}'.format(config['EXPERIMENT']))
        logging.info('config:\n {}'.format(json.dumps(config, indent=4, sort_keys=True)))
    
        try:
            # load model
            globals()['model'] = load_pretrained_model(config, metrics)
            model.summary()
        except Exception as e:
            logging.error(str(e))

interactive(children=(Dropdown(description='config_file', options=('reports/configs/3D/new_model/2019-09-05_11…

## Create Model

In [1]:
# create model
logging.info('Create model')
model = get_model(config, metrics)
model.summary()

NameError: name 'logging' is not defined

In [8]:
plot_model(model, to_file=os.path.join(config.get('CONFIG_PATH'),'model.png'), show_shapes=True)

In [ ]:
initial_epoch = 0
# training

# start a new main process for this training to free gpu memory afterwards
with tf.device(current_gpu):
    logging.info('Fit model, start trainings process')
    # fit model with trainingsgenerator
    results = model.fit_generator(
        generator=batch_generator,
        epochs=config['EPOCHS'],
        callbacks=get_callbacks(config, batch_generator, validation_generator),
        steps_per_epoch = len(batch_generator),
        validation_data=validation_generator,
        initial_epoch=initial_epoch,
        max_queue_size=30,
        workers=8,
        verbose=1)

2019-09-05 12:31:14,439 INFO Fit model, start trainings process
2019-09-05 12:31:15,246 INFO feed 4 Tensorboard is ready


Epoch 1/150
188/188 [==============================] - 137s 728ms/step - loss: 0.3455 - dice_coef_labels: 0.0227 - dice_coef_myo: 0.0152 - dice_coef_lv: 0.0151 - dice_coef_rv: 0.0363 - val_loss: 0.5302 - val_dice_coef_labels: 0.0253 - val_dice_coef_myo: 0.0164 - val_dice_coef_lv: 0.0134 - val_dice_coef_rv: 0.0432


2019-09-05 12:33:48,827 INFO Saved model to disk: models/3D/new_model/2019-09-05_12_31



Epoch 00001: val_dice_coef_labels improved from -inf to 0.02530, saving model to models/3D/new_model/2019-09-05_12_31/checkpoint.h5
Epoch 2/150
188/188 [==============================] - 116s 619ms/step - loss: -0.0019 - dice_coef_labels: 0.0974 - dice_coef_myo: 0.0708 - dice_coef_lv: 0.0743 - dice_coef_rv: 0.1312 - val_loss: 0.3768 - val_dice_coef_labels: 0.0676 - val_dice_coef_myo: 0.0480 - val_dice_coef_lv: 0.0476 - val_dice_coef_rv: 0.0882

Epoch 00002: val_dice_coef_labels improved from 0.02530 to 0.06758, saving model to models/3D/new_model/2019-09-05_12_31/checkpoint.h5
Epoch 3/150
188/188 [==============================] - 118s 629ms/step - loss: -0.3663 - dice_coef_labels: 0.3875 - dice_coef_myo: 0.2576 - dice_coef_lv: 0.3640 - dice_coef_rv: 0.4479 - val_loss: -0.3452 - val_dice_coef_labels: 0.3717 - val_dice_coef_myo: 0.3059 - val_dice_coef_lv: 0.2859 - val_dice_coef_rv: 0.4578

Epoch 00003: val_dice_coef_labels improved from 0.06758 to 0.37166, saving model to models/3D/new

## Write trainings history to disk

In [ ]:
logging.info(results.history)
df_history = pd.DataFrame(results.history)

df_history.to_csv(os.path.join('reports/history/3D_unet/', config['EXPERIMENT'] + '.csv'))
df_history.plot()

## Save model to disk

In [ ]:
from src.utils.utils_io import ensure_dir
model_json = model.to_json()
model_path = os.path.join('models/', config['EXPERIMENT'])
ensure_dir(model_path)
with open(os.path.join(model_path, 'model.json'), "w") as json_file:
    json_file.write(model_json)

# serialize weights to HDF5
name = 'weights_e-{0}_val_loss-{1}.h5'.format(config['EXPERIMENT'], '01')
model.save_weights(os.path.join(model_path, name))
logging.info("Saved model to disk: {}".format(model_path))

## Calculate metrics

In [ ]:
# predict for all batches
from medpy.metric.binary import hd, dc,jc,precision,recall
import numpy as np
gt_ = []
pred_ = []
img_ = []
n_batches = len(test_generator)
#n_batches = 2
logging.info('load and predict {} batches'.format(n_batches))
#with tf.device(current_gpu):
#    pred = model.predict_generator(batch_generator, steps = len(batch_generator))
#
#gt_ = [gt_.extend(batch[1]) for batch in batch_generator]
    


for idx,batch in enumerate(test_generator):
    if idx <= n_batches:
        with tf.device(current_gpu):
            img_.extend(batch[0])
            pred_.extend(((model.predict_on_batch(batch[0]))>=0.5).astype(np.bool))
        gt_.extend(batch[1])
    else:
        break

# reshape
gt = np.array(gt_)
pred = np.array(pred_)

logging.info('gt shape: {}'.format(gt.shape))
logging.info('pred shape: {}'.format(pred.shape))
del gt_
del pred_

In [ ]:
# calc medpy scores
jaccard_coef = jc(pred, gt)
dice_score = dc(pred, gt)
precision_score = precision(pred, gt)
recall_score = recall(pred, gt)

logging.info('jac: {}'.format(jaccard_coef))
logging.info('dice: {}'.format(dice_score))
logging.info('prec: {}'.format(precision_score))
logging.info('recall: {}'.format(recall_score))

In [ ]:
# calc IOU per channel
for c in range(pred.shape[-1]):
    pred_ = pred[...,c]
    gt_ = gt[...,c]
    # calc medpy scores
    jaccard_coef = jc(pred_, gt_)
    dice_score = dc(pred_, gt_)
    precision_score = precision(pred_, gt_)
    recall_score = recall(pred_, gt_)

    logging.info('jac: {}'.format(jaccard_coef))
    logging.info('dice: {}'.format(dice_score))
    logging.info('prec: {}'.format(precision_score))
    logging.info('recall: {}'.format(recall_score))



In [ ]:
from src.utils.my_metrics import jaccard_coef, jaccard_coef_background, jaccard_coef_rv, jaccard_coef_lv, jaccard_coef_myo, bce_dice_iou_loss, weighted_categorical_crossentropy, cce_dice_loss, weighted_cce_dice_coef

In [ ]:
# calc IOU per channel
for c in range(pred.shape[-1]):
    pred_ = pred[...,c]
    gt_ = gt[...,c]
    # calc medpy scores
    jaccard_coef = jc(pred_, gt_)
    dice_score = dc(pred_, gt_)
    precision_score = precision(pred_, gt_)
    recall_score = recall(pred_, gt_)

    logging.info('jac: {}'.format(jaccard_coef))
    logging.info('dice: {}'.format(dice_score))
    logging.info('prec: {}'.format(precision_score))
    logging.info('recall: {}'.format(recall_score))


In [ ]:
from src.visualization.visualize import plot_3d_vol

In [ ]:
plot_3d_vol(img_[0], gt[0])

In [ ]:
plot_3d_vol(img_[0], pred[0])

In [ ]:
print(pred_[0].shape)

In [ ]:
plot_3d_vol(pred_[0])

In [ ]:
pred[0].max()